In [1]:
import os, re 
import numpy as np
import tensorflow as tf
import glob
import os

## 데이터 전처리

### 1) 전처리: 데이터 로드
요구사항
* 12000개 이상 사전을 생성
* 텐서 길이는 15를 넘지 않도록
* 데이터상 필요없는 텍스트는 정제 할 것

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
raw_corpus = []

for txt_file in glob.glob(txt_file_path):
    with open(txt_file, "r") as f:
        raw_corpus.extend(f.readlines())

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord\n", 'That David played, and it pleased the Lord\n', "But you don't really care for music, do you?\n"]


### 2) 전처리: 데이터 정제

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # ?.!,¿ 기호 앞에 공백 추가
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 허용된 문자 외 공백 치환
    sentence = re.sub(r"\s+", " ", sentence) # 하나 이상 중복된 공백(스페이스) 하나로 처리
    return f"<start> { sentence.strip() } <end>" # start/end 마커 추가

# 임의문장 테스트
print("임의 문장: This @_is ;;;sample        sentence.")
print("정제:", preprocess_sentence("This @_is ;;;sample        sentence."))
print("----")

# 앞 단계에서 확인한 3개 문장 의도된 전처리여부 적용 확인
for i in range(3):
    print(f"before: \"{raw_corpus[i]}\"")
    print(f"after: \"{preprocess_sentence(raw_corpus[i])}\"")
    print("----")

임의 문장: This @_is ;;;sample        sentence.
정제: <start> this is sample sentence . <end>
----
before: "Now I've heard there was a secret chord
"
after: "<start> now i ve heard there was a secret chord <end>"
----
before: "That David played, and it pleased the Lord
"
after: "<start> that david played , and it pleased the lord <end>"
----
before: "But you don't really care for music, do you?
"
after: "<start> but you don t really care for music , do you ? <end>"
----


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 15 : continue  # 텐서 길이 15이상은 필터링
    corpus.append(preprocessed_sentence)

corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

> 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences 참고자료
>
> https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
>
> https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [5]:
def tokenize(corpus):
    # 12,500 단어장 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12500, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
print("텐서 길이:", len(tensor[0]))
assert len(tensor[0]) == 15  # 각 텐서길이 15개를 안넘도록 했으니

텐서 길이: 15


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6269    3    0    0    0]
[  50    4   95  303   62   53    9  946 6269    3    0    0    0    0]


> 데이터셋에 대해
>
> https://www.tensorflow.org/api_docs/python/tf/data/Dataset

## 모델 작성

In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [10]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12501), dtype=float32, numpy=
array([[[ 1.13146656e-04, -1.36220260e-04, -6.48452115e-05, ...,
         -1.84700664e-04, -1.47142600e-05, -1.71026004e-05],
        [ 3.57334095e-04,  7.55200090e-05, -6.53226380e-05, ...,
         -1.23574515e-04, -1.60609605e-04,  1.93327316e-04],
        [ 3.07683425e-04,  9.48408269e-05,  1.36184186e-04, ...,
         -8.28442498e-05, -1.02218139e-04,  3.69507703e-04],
        ...,
        [-2.16732486e-04, -1.55893981e-03,  2.44609360e-03, ...,
         -6.03584806e-04,  2.10143538e-04,  4.40827629e-04],
        [-1.27746607e-04, -1.87352661e-03,  2.77472334e-03, ...,
         -7.30416796e-04,  7.26693688e-05,  3.95436393e-04],
        [-2.75127168e-05, -2.17176485e-03,  3.03186802e-03, ...,
         -8.34000588e-04, -6.92017129e-05,  3.14347271e-04]],

       [[ 1.13146656e-04, -1.36220260e-04, -6.48452115e-05, ...,
         -1.84700664e-04, -1.47142600e-05, -1.71026004e-05],
        [ 7.06007049e-05, -1.16885873e-04, -6

In [11]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3200256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12813525  
Total params: 29,653,461
Trainable params: 29,653,461
Non-trainable params: 0
_________________________________________________________________


> optimizer와 loss등은 차차 배웁니다
>
> 혹시 미리 알고 싶다면 아래 문서를 참고하세요
>
> https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
>
> https://www.tensorflow.org/api_docs/python/tf/keras/losses
>
> 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다

## 모델 트레이닝 결과

In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True,
    reduction = 'none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
653/653 [==============================] - 119s 179ms/step - loss: 3.2130
Epoch 2/10
653/653 [==============================] - 117s 180ms/step - loss: 2.7999
Epoch 3/10
653/653 [==============================] - 117s 179ms/step - loss: 2.6473
Epoch 4/10
653/653 [==============================] - 117s 179ms/step - loss: 2.5276
Epoch 5/10
653/653 [==============================] - 117s 179ms/step - loss: 2.4273
Epoch 6/10
653/653 [==============================] - 117s 179ms/step - loss: 2.3378
Epoch 7/10
653/653 [==============================] - 117s 179ms/step - loss: 2.2537
Epoch 8/10
653/653 [==============================] - 117s 179ms/step - loss: 2.1756
Epoch 9/10
653/653 [==============================] - 117s 179ms/step - loss: 2.1023
Epoch 10/10
653/653 [==============================] - 117s 179ms/step - loss: 2.0327


## 모델이 만들어 낸 가사

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m gonna be a little selfish <end> '